# 🛡️ Snyk Multi-Branch Scanner for Azure DevOps (Windows Edition)

這個 Notebook 專門用來處理由 **Azure DevOps** 代管的 Python 專案。它會自動抓取所有遠端 Branch，並逐一執行 `pip install` 與 `snyk monitor`。

### 📋 前置需求
1. 確保已安裝 `git` 與 `snyk` CLI。
2. 確保目錄下有 `.env` 檔案，內容如下：
   ```ini
   ADO_REPO_URL=https://<PAT>@[dev.azure.com/Org/Proj/_git/Repo](https://dev.azure.com/Org/Proj/_git/Repo)
   LOCAL_REPO_PATH=C:\Temp\Snyk_Repo
   SNYK_TOKEN=你的_API_Token
   SNYK_ORG_ID=你的_Org_ID (可選)
   PROJECT_PREFIX=MyApp
   ```

In [ ]:
# Cell 1: 安裝必要套件與載入環境變數
import sys
import os
import subprocess
import time

# 自動安裝 python-dotenv (如果你還沒裝的話)
try:
    import dotenv
except ImportError:
    print("📦 Installing python-dotenv...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])
    import dotenv

from dotenv import load_dotenv

# 載入 .env
load_dotenv()

# 讀取變數
REPO_URL = os.getenv("ADO_REPO_URL")
LOCAL_PATH = os.getenv("LOCAL_REPO_PATH")
SNYK_TOKEN = os.getenv("SNYK_TOKEN")
SNYK_ORG_ID = os.getenv("SNYK_ORG_ID")
PROJECT_PREFIX = os.getenv("PROJECT_PREFIX", "SnykScan")

# 驗證
if not REPO_URL or not LOCAL_PATH:
    raise ValueError("❌ 缺少必要變數！請檢查 .env 檔案中的 ADO_REPO_URL 和 LOCAL_REPO_PATH。")

print(f"✅ 設定載入完成！")
print(f"📍 Target Repo: {REPO_URL}")
print(f"📂 Local Path: {LOCAL_PATH}")

In [ ]:
# Cell 2: 定義 Windows 專用指令執行函式 (處理編碼問題)
def run_command(command, cwd=None, env_vars=None):
    """
    執行 Shell 指令並回傳結果，包含錯誤處理與編碼相容性。
    """
    my_env = os.environ.copy()
    if env_vars:
        my_env.update(env_vars)

    try:
        # Windows 上 shell=True 是必須的，不然很多指令會找不到
        result = subprocess.run(
            command,
            cwd=cwd,
            shell=True,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            encoding='utf-8', # 嘗試 UTF-8
            errors='replace', # 如果 Windows 跳 cp950 亂碼，用 replace 忽略錯誤
            env=my_env
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"❌ Command Failed: {command}")
        print(f"🔻 Stderr: {e.stderr}")
        return None

print("✅ Helper function ready.")

In [ ]:
# Cell 3: 檢查工具與 Snyk 登入
print("🔍 Checking Environment...")

# Check Git
git_ver = run_command("git --version")
if git_ver:
    print(f"🔹 {git_ver}")
else:
    raise RuntimeError("❌ Git not found! Please install Git for Windows.")

# Check Snyk
snyk_ver = run_command("snyk --version")
if snyk_ver:
    print(f"🔹 Snyk CLI: {snyk_ver}")
else:
    raise RuntimeError("❌ Snyk CLI not found! Run 'npm install -g snyk' or download the binary.")

# Authenticate
if SNYK_TOKEN:
    print("🔐 Authenticating Snyk with Token...")
    # 使用 snyk auth <token> 進行授權
    run_command(f"snyk auth {SNYK_TOKEN}")
else:
    print("⚠️ No SNYK_TOKEN in .env. Assuming you are already logged in.")

In [ ]:
# Cell 4: 下載或更新 Repo
if os.path.exists(LOCAL_PATH):
    print(f"📂 Folder exists at {LOCAL_PATH}. Fetching updates...")
    # 強制 fetch 確保拿到最新的 branch 列表
    run_command("git fetch --all", cwd=LOCAL_PATH)
else:
    print(f"🚀 Cloning repo to {LOCAL_PATH}...")
    # 如果 REPO_URL 包含 PAT，這裡就會直接 Clone 成功
    run_command(f"git clone {REPO_URL} {LOCAL_PATH}")

print("✅ Git Repo ready for scanning.")

In [ ]:
# Cell 5: 解析遠端 Branch 清單
print("🔍 Parsing remote branches...")

# 取得所有遠端分支
raw_branches = run_command("git branch -r", cwd=LOCAL_PATH)
branches = []

if raw_branches:
    for line in raw_branches.splitlines():
        b = line.strip()
        # 過濾掉 '->' 指標和 origin 前綴
        if "->" not in b and "origin/" in b:
            clean_name = b.replace("origin/", "")
            branches.append(clean_name)

print(f"📊 Found {len(branches)} branches: {branches}")

In [ ]:
# Cell 6: 批次掃描迴圈 (The Main Loop)

print("🚀 Starting Snyk Monitor Batch Job...")

for i, branch in enumerate(branches):
    print(f"\n------------------------------------------------------")
    print(f"[{i+1}/{len(branches)}] 👉 Processing Branch: {branch}")
    
    # 1. Checkout (強制 Clean 以免衝突)
    run_command("git reset --hard", cwd=LOCAL_PATH)
    # 注意: 如果有 untracked files 導致 checkout 失敗，可加 git clean -fdx (小心使用)
    
    # Windows Path 處理：確保 branch 名稱沒有奇怪字元
    checkout_res = run_command(f"git checkout {branch}", cwd=LOCAL_PATH)
    run_command(f"git pull origin {branch}", cwd=LOCAL_PATH)
    
    if checkout_res is None:
        print(f"❌ Failed to checkout {branch}, skipping.")
        continue

    # 2. 安裝依賴 (使用目前的 Jupyter Python Kernel)
    req_path = os.path.join(LOCAL_PATH, "requirements.txt")
    if os.path.exists(req_path):
        print("📦 Installing requirements.txt...")
        # -q 安靜模式，避免 log 爆炸
        run_command(f"\"{sys.executable}\" -m pip install -r requirements.txt -q", cwd=LOCAL_PATH)
    else:
        print("⚠️ No requirements.txt found. Snyk might report inaccurate results.")

    # 3. 執行 Snyk Monitor
    # 建構 Project Name: e.g. "MyApp-feature-login"
    proj_name = f"{PROJECT_PREFIX}-{branch}"
    
    # Snyk 指令
    cmd = f"snyk monitor --target-reference=\"{branch}\" --project-name=\"{proj_name}\""
    if SNYK_ORG_ID:
        cmd += f" --org={SNYK_ORG_ID}"

    print(f"🛡️ Sending snapshot to Snyk (Project: {proj_name})...")
    snyk_out = run_command(cmd, cwd=LOCAL_PATH)

    if snyk_out and "Explore this snapshot" in snyk_out:
        print(f"✅ Success! Snapshot uploaded for {branch}")
    else:
        print(f"❌ Snyk Monitor failed for {branch}")
        if snyk_out:
            # 印出最後 300 字元除錯
            print(f"Debug Log: ...{snyk_out[-300:]}")
            
    # 休息 2 秒避免 API Rate Limit
    time.sleep(2)

print("\n🎉 All branches processed! (記得去清空 Temp 資料夾喔)")